# 🏦💱 Hybrid OpenAPI Tools + Semantic Kernel Plugins Tutorial

**🚀 Learn how to combine OpenAPI tools with Semantic Kernel plugins for powerful Azure AI Agents!**

This tutorial demonstrates:
1. **OpenAPI Tools** - External currency exchange API
2. **Semantic Kernel Plugins** - Local banking functions
3. **Hybrid Agent** - Combining both for rich conversational experiences

**Real-world scenario:** A banking assistant that can:
- 💰 Check account balances and transactions
- 💱 Get current exchange rates
- 🌍 Calculate international transfer amounts
- 📊 Provide financial insights combining local and external data

---

## 🔧 Setup and Prerequisites

**Environment Variables Required:**
- `PROJECT_ENDPOINT`: Your Azure AI Project endpoint
- `MODEL_DEPLOYMENT_NAME`: Your deployed AI model name

**Architecture Overview:**
```
Azure AI Agent
    ├── OpenAPI Tool → Frankfurter API (currency exchange)
    └── SK Plugins
            ├── BankingPlugin → get_account_balances()
            └── TransactionPlugin → get_recent_transactions()
```

In [1]:
# Install required packages
# !pip install azure-ai-agents azure-identity semantic-kernel

import os
import json
from typing import Annotated
from datetime import datetime, timedelta

# Azure AI Agents imports
from semantic_kernel import Kernel
from semantic_kernel.filters import AutoFunctionInvocationContext, FilterTypes
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails
from azure.identity import DefaultAzureCredential
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import AuthorRole

print("✅ All packages imported successfully!")

✅ All packages imported successfully!


---

# Part 1: Sample Banking Data

Let's start by setting up our sample banking data that our plugins will use.

In [2]:
# Sample bank accounts data
SAMPLE_ACCOUNTS = [
    {
        "account_id": "acc_001",
        "account_name": "Checking Account",
        "account_type": "checking",
        "currency": "USD",
        "balance": 6233.59,
        "available_balance": 6233.59
    },
    {
        "account_id": "acc_002",
        "account_name": "Savings Account",
        "account_type": "savings",
        "currency": "USD",
        "balance": 15750.00,
        "available_balance": 15750.00
    },
    {
        "account_id": "acc_003",
        "account_name": "EUR Travel Account",
        "account_type": "checking",
        "currency": "EUR",
        "balance": 2500.00,
        "available_balance": 2500.00
    }
]

# Sample transaction data
SAMPLE_TRANSACTIONS = [
    {
        "id": "txn_001",
        "account_id": "acc_001",
        "date": "2024-12-01",
        "description": "Grocery Store Purchase",
        "amount": -85.47,
        "balance": 2914.53,
        "category": "Food & Dining"
    },
    {
        "id": "txn_002",
        "account_id": "acc_001",
        "date": "2024-12-02",
        "description": "Salary Deposit",
        "amount": 3500.00,
        "balance": 6414.53,
        "category": "Income"
    },
    {
        "id": "txn_003",
        "account_id": "acc_001",
        "date": "2024-12-03",
        "description": "Gas Station",
        "amount": -45.20,
        "balance": 6369.33,
        "category": "Transportation"
    },
    {
        "id": "txn_004",
        "account_id": "acc_001",
        "date": "2024-12-04",
        "description": "Online Shopping",
        "amount": -129.99,
        "balance": 6239.34,
        "category": "Shopping"
    },
    {
        "id": "txn_005",
        "account_id": "acc_001",
        "date": "2024-12-05",
        "description": "Coffee Shop",
        "amount": -5.75,
        "balance": 6233.59,
        "category": "Food & Dining"
    },
    {
        "id": "txn_006",
        "account_id": "acc_002",
        "date": "2024-12-01",
        "description": "Interest Payment",
        "amount": 25.00,
        "balance": 15750.00,
        "category": "Income"
    },
    {
        "id": "txn_007",
        "account_id": "acc_003",
        "date": "2024-11-28",
        "description": "Paris Hotel",
        "amount": -150.00,
        "balance": 2500.00,
        "category": "Travel"
    }
]

print("📊 Sample banking data loaded:")
print(f"   - {len(SAMPLE_ACCOUNTS)} accounts")
print(f"   - {len(SAMPLE_TRANSACTIONS)} transactions")
print(f"   - Total USD balance: ${sum(acc['balance'] for acc in SAMPLE_ACCOUNTS if acc['currency'] == 'USD'):,.2f}")

📊 Sample banking data loaded:
   - 3 accounts
   - 7 transactions
   - Total USD balance: $21,983.59


---

# Part 2: Create Semantic Kernel Banking Plugins

Now we'll create two Semantic Kernel plugins that provide banking functionality:
1. **BankingPlugin** - Account balances and summaries
2. **TransactionPlugin** - Transaction history and analysis

In [3]:
class BankingPlugin:
    """Banking plugin that provides account balance and summary information."""
    
    @kernel_function(
        description="Get all account balances for the user"
    )
    def get_account_balances(self) -> Annotated[str, "Returns account balances in JSON format"]:
        """Get current balances for all user accounts."""
        balance_info = []
        total_usd = 0
        
        for account in SAMPLE_ACCOUNTS:
            balance_info.append({
                "account_name": account["account_name"],
                "account_type": account["account_type"],
                "currency": account["currency"],
                "balance": account["balance"],
                "available_balance": account["available_balance"]
            })
            
            # Sum USD accounts for total
            if account["currency"] == "USD":
                total_usd += account["balance"]
        
        result = {
            "accounts": balance_info,
            "summary": {
                "total_accounts": len(SAMPLE_ACCOUNTS),
                "total_usd_balance": total_usd
            }
        }
        
        return json.dumps(result, indent=2)
    
    @kernel_function(
        description="Get balance for a specific account by name or type"
    )
    def get_specific_account_balance(
        self, 
        account_identifier: Annotated[str, "Account name or type (e.g., 'checking', 'savings', 'EUR Travel Account')"]
    ) -> Annotated[str, "Returns specific account balance information"]:
        """Get balance for a specific account."""
        identifier_lower = account_identifier.lower()
        
        for account in SAMPLE_ACCOUNTS:
            if (identifier_lower in account["account_name"].lower() or 
                identifier_lower == account["account_type"].lower()):
                return json.dumps({
                    "account_name": account["account_name"],
                    "account_type": account["account_type"],
                    "currency": account["currency"],
                    "balance": account["balance"],
                    "available_balance": account["available_balance"]
                }, indent=2)
        
        return json.dumps({
            "error": f"Account '{account_identifier}' not found",
            "available_accounts": [acc["account_name"] for acc in SAMPLE_ACCOUNTS]
        }, indent=2)

print("🏦 BankingPlugin created successfully!")

🏦 BankingPlugin created successfully!


In [4]:
class TransactionPlugin:
    """Transaction plugin that provides transaction history and analysis."""
    
    @kernel_function(
        description="Get recent transactions for all accounts or a specific account"
    )
    def get_recent_transactions(
        self,
        limit: Annotated[int, "Number of transactions to return (default 5)"] = 5,
        account_id: Annotated[str, "Specific account ID (optional)"] = None
    ) -> Annotated[str, "Returns recent transactions in JSON format"]:
        """Get recent transactions, optionally filtered by account."""
        transactions = SAMPLE_TRANSACTIONS.copy()
        
        # Filter by account if specified
        if account_id:
            transactions = [t for t in transactions if t["account_id"] == account_id]
        
        # Sort by date (most recent first) and limit
        transactions.sort(key=lambda x: x["date"], reverse=True)
        transactions = transactions[:limit]
        
        # Add account names for context
        for transaction in transactions:
            account = next((acc for acc in SAMPLE_ACCOUNTS if acc["account_id"] == transaction["account_id"]), None)
            if account:
                transaction["account_name"] = account["account_name"]
        
        result = {
            "transactions": transactions,
            "summary": {
                "count": len(transactions),
                "total_amount": sum(t["amount"] for t in transactions)
            }
        }
        
        return json.dumps(result, indent=2)
    
    @kernel_function(
        description="Get spending analysis by category"
    )
    def get_spending_by_category(self) -> Annotated[str, "Returns spending breakdown by category"]:
        """Analyze spending patterns by category."""
        category_spending = {}
        
        for transaction in SAMPLE_TRANSACTIONS:
            if transaction["amount"] < 0:  # Only count expenses (negative amounts)
                category = transaction["category"]
                if category not in category_spending:
                    category_spending[category] = 0
                category_spending[category] += abs(transaction["amount"])
        
        # Sort by spending amount
        sorted_categories = sorted(category_spending.items(), key=lambda x: x[1], reverse=True)
        
        result = {
            "spending_by_category": [
                {"category": cat, "total_spent": amount} 
                for cat, amount in sorted_categories
            ],
            "total_expenses": sum(category_spending.values()),
            "top_category": sorted_categories[0][0] if sorted_categories else None
        }
        
        return json.dumps(result, indent=2)
    
    @kernel_function(
        description="Calculate available funds for international transfer"
    )
    def calculate_transfer_capacity(
        self,
        source_currency: Annotated[str, "Source currency code (e.g., USD, EUR)"] = "USD"
    ) -> Annotated[str, "Returns available transfer amounts by currency"]:
        """Calculate how much is available for international transfers."""
        available_funds = {}
        
        for account in SAMPLE_ACCOUNTS:
            currency = account["currency"]
            if currency not in available_funds:
                available_funds[currency] = 0
            available_funds[currency] += account["available_balance"]
        
        # Focus on requested currency
        if source_currency in available_funds:
            main_amount = available_funds[source_currency]
        else:
            main_amount = 0
        
        result = {
            "requested_currency": source_currency,
            "available_amount": main_amount,
            "all_currencies": available_funds,
            "transfer_ready": main_amount > 0
        }
        
        return json.dumps(result, indent=2)

print("💳 TransactionPlugin created successfully!")

💳 TransactionPlugin created successfully!


---

# Part 3: Load Currency Exchange OpenAPI Tool

Now let's load the currency exchange OpenAPI specification that we'll combine with our banking plugins.

In [6]:
# Load the currency exchange OpenAPI specification
currency_spec_path = os.path.join("openapi_files", "currency_exchange.json")

try:
    with open(currency_spec_path, "r") as f:
        currency_openapi_spec = json.loads(f.read())
    
    print("💱 Currency exchange OpenAPI spec loaded successfully!")
    print(f"   - API Title: {currency_openapi_spec['info']['title']}")
    print(f"   - API Version: {currency_openapi_spec['info']['version']}")
    print(f"   - Server: {currency_openapi_spec['servers'][0]['url']}")
    
except FileNotFoundError:
    print("⚠️ Currency exchange OpenAPI file not found!")
    print("   Creating a simplified version for demo purposes...")
    
    # Fallback: create a simplified spec
    currency_openapi_spec = {
        "openapi": "3.0.1",
        "info": {
            "title": "Frankfurter Exchange Rates API",
            "description": "Returns the latest foreign exchange reference rates.",
            "version": "1.0"
        },
        "servers": [
            {"url": "https://api.frankfurter.dev"}
        ],
        "paths": {
            "/v1/latest": {
                "get": {
                    "summary": "Latest Exchange Rates",
                    "description": "Returns the latest exchange rates relative to the base currency.",
                    "operationId": "get-latest-rates",
                    "parameters": [
                        {
                            "in": "query",
                            "name": "base",
                            "schema": {"type": "string", "example": "USD"},
                            "description": "The base currency (e.g., USD, EUR)"
                        }
                    ],
                    "responses": {
                        "200": {
                            "description": "Successful response",
                            "content": {
                                "application/json": {
                                    "example": {
                                        "amount": 1,
                                        "base": "USD",
                                        "date": "2024-12-05",
                                        "rates": {
                                            "EUR": 0.88,
                                            "GBP": 0.75,
                                            "JPY": 143.5
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
    
    print("✅ Fallback currency exchange spec created!")

💱 Currency exchange OpenAPI spec loaded successfully!
   - API Title: Frankfurter Exchange Rates API
   - API Version: 1.0
   - Server: https://api.frankfurter.dev


---

# Part 4: Create Hybrid Azure AI Agent

Now we'll create an Azure AI Agent that combines:
- **OpenAPI Tool** for currency exchange
- **Semantic Kernel Plugins** for banking operations

This demonstrates the power of hybrid agent architectures!

In [7]:
# Defining a Kernel Filter
kernel = Kernel()

# Register the auto function invocation filter
@kernel.filter(FilterTypes.AUTO_FUNCTION_INVOCATION)
async def auto_function_invocation_filter(context: AutoFunctionInvocationContext, next):
    """A filter that will be called for each function call in the response."""
    print(80*"=")
    print("\n🔧 Auto function invocation filter - Local Plugins Only (doesnt detect Azure AI Agents OpenAPI function calls)")
    print(f"📞 Function: {context.function.name}")
    print("📝 Arguments:", context.arguments)
    print(f"🔢 Request sequence: {context.request_sequence_index}")
    print(f"🎯 Function sequence: {context.function_sequence_index}")

    # as an example
    try:
        function_calls = context.chat_history.messages[-1].items
        print(f"📊 Number of function calls: {len(function_calls)}")
    except:
        pass
    # if we don't call next, it will skip this function, and go to the next one
    await next(context)
    
    print("📤 Function Call Contents:", context.function_call_content)
    print("✅ Function Call Result:", context.function_result)
    print(80*"=")



# Create the hybrid banking agent with OpenAPI tools and Semantic Kernel plugins
async def create_hybrid_banking_agent():
    """Create Azure AI Agent with both OpenAPI tools and SK plugins."""
    
    # Step 1: Create the Azure AI Agents client
    agents_client = AgentsClient(
        endpoint=os.environ["PROJECT_ENDPOINT"],
        credential=DefaultAzureCredential()
    )
    
    print("🔗 Connected to Azure AI Agents service")
    
    # Step 2: Create OpenAPI tool for currency exchange
    auth = OpenApiAnonymousAuthDetails()
    currency_tool = OpenApiTool(
        name="currency_exchange",
        spec=currency_openapi_spec,
        description="Get the latest foreign exchange rates from Frankfurter API. Use this for currency conversion and international transfer calculations.",
        auth=auth
    )
    
    print("💱 Currency exchange OpenAPI tool created")
    
    # Step 3: Create Azure AI agent with OpenAPI tool
    base_agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="hybrid_banking_agent",
        instructions="""You are an expert banking and financial assistant with access to both banking data and currency exchange rates.
        
You can help users with:
1. Account balances and banking information
2. Transaction history and spending analysis
3. Currency exchange rates and conversion calculations
4. International transfer planning and cost estimation

Always provide clear, accurate financial information and explain calculations step by step.
When dealing with international transfers, always get current exchange rates and calculate both the amount to be sent and received.
Be helpful and proactive in suggesting related financial insights.""",
        tools=currency_tool.definitions
    )
    
    print(f"🏦 Base Azure AI agent created: {base_agent.id}")
    
    # Step 4: Create Semantic Kernel AzureAIAgent wrapper with plugins
    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=os.environ["PROJECT_ENDPOINT"]) as project_client,
    ):
        # Wrap the existing agent with SK and add plugins
        hybrid_agent = AzureAIAgent(
            client=project_client,
            definition=base_agent,
            kernel=kernel,
            plugins=[BankingPlugin(), TransactionPlugin()]  # Add our local plugins!
        )
        
        print("🚀 Hybrid agent created with both OpenAPI tools and SK plugins!")
        
        return hybrid_agent, agents_client, base_agent

# Create the hybrid agent
print("Creating hybrid banking agent...")
# Note: We'll actually run this in the next cell due to async context

Creating hybrid banking agent...


---

# Part 5: Test Hybrid Agent with Complex Scenarios

Let's test our hybrid agent with queries that require both local banking data and external currency exchange information.

In [16]:
async def test_hybrid_agent():
    """Test the hybrid agent with complex scenarios."""
    
    # Create SK wrapper with plugins
    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=os.environ["PROJECT_ENDPOINT"]) as project_client,
    ):
        hybrid_agent, agents_client, base_agent = await create_hybrid_banking_agent()

        print("🚀 Hybrid Banking Agent Ready!")
        print("   ✅ OpenAPI Tool: Currency Exchange (Frankfurter API)")
        print("   ✅ SK Plugin: Banking Operations")
        print("   ✅ SK Plugin: Transaction Analysis")
        
        # Test scenarios that combine multiple tools
        test_scenarios = [
            {
                "name": "Currency Conversion",
                "query": "What is the current exchange rate from USD to EUR? How much EUR can I get for 1000 USD?"
            },
            {
                "name": "Account Summary",
                "query": "Give me a summary of all my account balances and my recent spending patterns."
            },
            {
                "name": "International Transfer Planning", 
                "query": "I want to send 1000 USD to Europe. Check my USD balance, get the current USD to EUR exchange rate, and tell me how much EUR they would receive."
            },
            {
                "name": "Multi-Currency Analysis",
                "query": "Show me all my account balances and convert my EUR account balance to USD using current exchange rates. Also analyze my spending by category."
            },
            {
                "name": "Travel Budget Planning",
                "query": "I'm planning a trip to Japan. Check my USD accounts, get the current USD to JPY exchange rate, and tell me how much JPY I could get with 500 USD. Also show me my travel-related expenses."
            }
        ]
        
        thread = None
        
        try:
            for i, scenario in enumerate(test_scenarios, 1):
                print(f"\n{'='*60}")
                print(f"🧪 Test Scenario {i}: {scenario['name']}")
                print(f"{'='*60}")
                print(f"👤 User Query: {scenario['query']}")
                print("\n🤖 Agent Response:")

                async for response in hybrid_agent.invoke(
                    messages=scenario['query'],
                    thread=thread,
                ):
                    if response.role != AuthorRole.TOOL:
                        print(response.content)
                    thread = response.thread
                
                print("\n" + "-"*40)
        
        finally:
            # Cleanup
            pass
    
    # Cleanup the base agent
    agents_client.delete_agent(base_agent.id)
    print("🧹 Agent cleaned up")
    print("\n✅ All hybrid agent tests completed!")

# Run the hybrid agent tests
await test_hybrid_agent()

🔗 Connected to Azure AI Agents service
💱 Currency exchange OpenAPI tool created
🏦 Base Azure AI agent created: asst_5QYG5CV02YpEg8Pap2tVB5yG
🚀 Hybrid agent created with both OpenAPI tools and SK plugins!
🚀 Hybrid Banking Agent Ready!
   ✅ OpenAPI Tool: Currency Exchange (Frankfurter API)
   ✅ SK Plugin: Banking Operations
   ✅ SK Plugin: Transaction Analysis

🧪 Test Scenario 1: Currency Conversion
👤 User Query: What is the current exchange rate from USD to EUR? How much EUR can I get for 1000 USD?

🤖 Agent Response:
The current exchange rate from USD to EUR is 1 USD = 0.85441 EUR.

For 1,000 USD, you will get:
1,000 USD × 0.85441 = 854.41 EUR

So, you can get approximately 854.41 EUR for 1,000 USD at the current exchange rate. If you need help factoring in transfer fees or want to know rates for another amount, let me know!

----------------------------------------

🧪 Test Scenario 2: Account Summary
👤 User Query: Give me a summary of all my account balances and my recent spending patt

---

# Part 6: Advanced Multi-Tool Query Examples

Let's create some specific examples that showcase the power of combining local plugins with external APIs.

In [15]:
async def demo_advanced_scenarios():
    """Demonstrate advanced scenarios combining multiple tools."""
    
    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds, endpoint=os.environ["PROJECT_ENDPOINT"]) as project_client,
    ):
        hybrid_agent, agents_client, base_agent = await create_hybrid_banking_agent()
        
        # Advanced demo scenarios
        advanced_queries = [
            """I'm considering a large international transfer. Can you:
            1. Check my current USD balance in all my accounts
            2. Get current USD to GBP exchange rates
            3. Calculate how much GBP I'd get for $2000
            4. Analyze if this transfer fits my spending patterns
            """,

            """Financial health check: Show me my account balances, recent spending by category, 
            and calculate the total value of all my accounts in USD using current exchange rates.
            """,
            
            """I want to optimize my EUR account. Check my EUR balance, 
            get current EUR to USD rates, and tell me if I should convert some EUR to USD 
            based on my spending patterns.
            """
        ]
        
        thread = None
        
        try:
            for i, query in enumerate(advanced_queries, 1):
                print(f"\n{'🎯' + '='*70}")
                print(f"Advanced Demo {i}: Multi-Tool Financial Analysis")
                print(f"{'='*71}")
                print(f"👤 Complex Query:")
                print(f"{query}")
                print("\n🤖 Comprehensive Analysis:")
                print("-" * 50)
                
                async for response in hybrid_agent.invoke(messages=query, thread=thread):
                    if response.role != AuthorRole.TOOL:
                        print(response.content)
                    thread = response.thread
                
                print("\n" + "="*71)
        
        finally:
            pass
    
    agents_client.delete_agent(base_agent.id)
    print("\n🚀 Advanced demo completed! The agent successfully combined:")
    print("   💰 Local banking data (balances, transactions)")
    print("   💱 Live currency exchange rates")
    print("   📊 Financial analysis and insights")

# Run advanced demos
print("Starting advanced multi-tool demos...")
await demo_advanced_scenarios()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B9B1A0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B99400>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078C28B90>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078C29580>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078BA5370>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B26DB0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078C37B90>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B9B230>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x0000017079127A70>, 164982.437)])']
connector: <aiohttp.connector.TCPConnector object at 0x00000170788CFE60>


Starting advanced multi-tool demos...
🔗 Connected to Azure AI Agents service
💱 Currency exchange OpenAPI tool created


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B95010>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000017078B94BF0>


🏦 Base Azure AI agent created: asst_pVpo8zenRU4ENBO2EDMkcPJF
🚀 Hybrid agent created with both OpenAPI tools and SK plugins!

🎯======================================================================
Advanced Demo 1: Multi-Tool Financial Analysis
👤 Complex Query:
I'm considering a large international transfer. Can you:
            1. Check my current USD balance in all my accounts
            2. Get current USD to GBP exchange rates
            3. Calculate how much GBP I'd get for $2000
            4. Analyze if this transfer fits my spending patterns
            

🤖 Comprehensive Analysis:
--------------------------------------------------
I'm unable to access your account or transaction data right now due to a system issue, which prevents a real-time balance check, spending analysis, and live calculation of transfer capacity. However, I can still provide guidance on how each step would work and give you information about current USD to GBP exchange rates if you'd like.

Here’s how I wo

---

## 🎯 Key Architecture Insights

### 🏗️ Hybrid Agent Architecture Benefits:

**1. OpenAPI Tools (External APIs):**
- ✅ **Real-time data**: Live currency rates, market data
- ✅ **Third-party services**: Leverage external expertise
- ✅ **No local maintenance**: APIs maintained by providers
- ✅ **Standardized**: OpenAPI spec ensures compatibility

**2. Semantic Kernel Plugins (Local Functions):**
- ✅ **Private data access**: Secure internal systems
- ✅ **Custom business logic**: Tailored to your needs
- ✅ **No external dependencies**: Reliable offline operation
- ✅ **Full control**: Complete customization possible

**3. Combined Power:**
- 🚀 **Rich experiences**: Internal + external data
- 🚀 **Intelligent routing**: Agent chooses right tools
- 🚀 **Complex workflows**: Multi-step operations
- 🚀 **Contextual responses**: Comprehensive insights

---

## 🔧 Implementation Best Practices

### ✅ Plugin Design:
- **Clear descriptions**: Help the agent understand when to use each function
- **Appropriate return types**: Use JSON for structured data
- **Error handling**: Gracefully handle missing data or invalid inputs
- **Focused responsibility**: Each plugin should have a clear purpose

### ✅ OpenAPI Integration:
- **Descriptive tool names**: Help agent understand API capabilities
- **Authentication handling**: Properly configure auth details
- **Rate limiting awareness**: Consider API usage limits
- **Error resilience**: Handle API failures gracefully

### ✅ Agent Instructions:
- **Clear capabilities**: Explain what the agent can do
- **Usage guidance**: Provide examples of how to use tools
- **Response formatting**: Specify how to present information
- **Context preservation**: Maintain conversation context across tool calls

---

## 🎨 Exercise: Create Your Own Hybrid Agent

**Challenge:** Extend this example by adding new capabilities!

### 💡 Ideas to Implement:

1. **Weather-Based Spending Plugin**:
   - Create a plugin that analyzes spending patterns by weather
   - Combine with a weather API to provide seasonal insights

2. **Investment Portfolio Plugin**:
   - Add stock/crypto portfolio management
   - Combine with financial market APIs

3. **Travel Expense Optimizer**:
   - Create plugins for travel planning
   - Combine with flight/hotel APIs and currency exchange

### 🧪 Try it yourself:

In [ ]:
# Your turn! Create a new plugin and combine it with an external API

class CustomPlugin:
    """Create your own plugin here!"""
    
    @kernel_function(
        description="Describe what your function does"
    )
    def your_custom_function(self) -> Annotated[str, "Return type description"]:
        """Implement your custom functionality here."""
        # Your code here
        return "Your implementation"

# Test your plugin
# custom_plugin = CustomPlugin()
# result = custom_plugin.your_custom_function()
# print(result)

print("🎨 Ready for your creative implementation!")
print("💡 Tip: Start simple, then add complexity gradually")
print("🚀 Remember: The power is in combining local + external data!")

<details>
<summary>💡 Click for Exercise Solution Ideas</summary>

**Sample Weather Plugin:**
```python
class WeatherSpendingPlugin:
    @kernel_function(description="Analyze spending patterns by weather conditions")
    def analyze_weather_spending(self) -> str:
        # Categorize transactions by likely weather impact
        weather_categories = {
            "indoor": ["Shopping", "Food & Dining"],
            "outdoor": ["Transportation", "Travel"],
            "seasonal": ["Utilities", "Clothing"]
        }
        # Your analysis logic here
        return json.dumps({"weather_impact_analysis": "Your insights"})
```

**Sample Investment Plugin:**
```python
class InvestmentPlugin:
    @kernel_function(description="Get portfolio summary")
    def get_portfolio_summary(self) -> str:
        # Mock portfolio data
        portfolio = {
            "stocks": {"AAPL": 10, "MSFT": 5},
            "total_value_usd": 15000,
            "performance": "+5.2%"
        }
        return json.dumps(portfolio)
```

</details>

---

## 🎯 Summary: Hybrid Agent Architecture

### 🏆 What We've Accomplished:

✅ **Created Semantic Kernel Plugins** for banking operations
✅ **Integrated OpenAPI Tools** for real-time currency exchange
✅ **Built Hybrid Agent** combining local + external capabilities
✅ **Demonstrated Complex Queries** requiring multiple tool calls
✅ **Showed Best Practices** for plugin design and integration

### 🚀 Key Takeaways:

1. **Plugin Design**: Create focused, well-documented functions
2. **OpenAPI Integration**: Leverage external APIs for real-time data
3. **Agent Instructions**: Provide clear guidance on tool usage
4. **Complex Workflows**: Enable multi-step operations across tools
5. **User Experience**: Create conversational, intelligent interactions

### 🔮 Next Steps:

- **Explore Authentication**: Add secured OpenAPI tools
- **Scale Plugin Architecture**: Create plugin ecosystems
- **Add Error Handling**: Robust production-ready implementations
- **Performance Optimization**: Efficient tool selection and execution
- **Multi-Agent Orchestration**: Coordinate multiple specialized agents

**🎉 Congratulations!** You've mastered hybrid agent architectures combining the best of local plugins and external APIs!

---

*Ready to build the next generation of intelligent financial assistants? The possibilities are endless!* 🌟